In [8]:
# hyperparameters ref
'''
    @article{ZHENG2023121607,
    title = {Interpretable building energy consumption forecasting using spectral clustering algorithm and temporal fusion transformers architecture},
    journal = {Applied Energy},
    volume = {349},
    pages = {121607},
    year = {2023},
    issn = {0306-2619},
    doi = {https://doi.org/10.1016/j.apenergy.2023.121607},
    url = {https://www.sciencedirect.com/science/article/pii/S0306261923009716},
    author = {Peijun Zheng and Heng Zhou and Jiang Liu and Yosuke Nakanishi},
    keywords = {Building energy consumption forecasting, Attention mechanism, Interpretable decomposition method, Interpretable deep learning model},
    abstract = {Accurate building energy consumption forecasting is crucial for developing efficient building energy management systems, improving energy efficiency, and local building energy supervision and management. However, short-term building energy consumption forecasting is challenging due to highly non-smooth and volatile trends. In this paper, we present a novel methodology that combines interpretable decomposition methods with an interpretable forecasting model. We first illustrate a daily energy consumption pattern recognition (DECPR) method, which decomposes daily energy consumption patterns into interpretable energy consumption subsequences. To achieve satisfactory forecasting performance, we design the vector representation of each subsequence as a static input to the temporal fusion transformers (TFT) model. This vector representation integrates the DECPR method into the TFT model. The TFT model produces interpretable outputs, such as the attention analysis of different step lengths and the visualization of the importance ranking of exogenous variables, including meteorological data, calendar information, and the vector representation. Empirical studies demonstrate that our proposed DECPR-TFT system outperforms comparable models with a mean absolute percentage error (MAPE) of 6.11%, which is significantly lower than other models. These interpretable outputs provide valuable insights for researchers seeking to develop energy-saving operation strategies in buildings. Overall, our methodology offers a promising solution for short-term building energy consumption forecasting that can contribute to more efficient building energy management and energy-saving operation strategies.}
    }
'''

'\n    @article{ZHENG2023121607,\n    title = {Interpretable building energy consumption forecasting using spectral clustering algorithm and temporal fusion transformers architecture},\n    journal = {Applied Energy},\n    volume = {349},\n    pages = {121607},\n    year = {2023},\n    issn = {0306-2619},\n    doi = {https://doi.org/10.1016/j.apenergy.2023.121607},\n    url = {https://www.sciencedirect.com/science/article/pii/S0306261923009716},\n    author = {Peijun Zheng and Heng Zhou and Jiang Liu and Yosuke Nakanishi},\n    keywords = {Building energy consumption forecasting, Attention mechanism, Interpretable decomposition method, Interpretable deep learning model},\n    abstract = {Accurate building energy consumption forecasting is crucial for developing efficient building energy management systems, improving energy efficiency, and local building energy supervision and management. However, short-term building energy consumption forecasting is challenging due to highly non-smooth

In [9]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from darts.datasets import WeatherDataset
from darts.models import LinearRegressionModel
import darts.metrics as metrics
from darts.datasets import AirPassengersDataset
import datetime

import numpy as np
import optuna
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.models import TCNModel
from darts.utils.likelihood_models import GaussianLikelihood
from optuna.terminator import report_cross_validation_scores

from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from darts import TimeSeries

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold




In [10]:
class Model_base():
    # set basic settings:
    # 1. model identification
    # 2. params of model, and whether call optuna
    # 3. prediction settings
    # 2. saving settings
    def __init__(self,id, model_type,
                 call_optuna, optuna_settings, optuna_params_dic,
                 call_one_trial, one_trial_settings, one_trial_params_dic,
                 load_from_trained_model,load_settings,
                 load_from_checkpoint,load_checkpoint_settings,
                 predict_settings,metric_settings, save_settings):
        # for optuna params_dic should be like:
        #   dic={
        #       'param_key':[tpye, lower_bound, upper_bound, bool_log], #params to search
        #       'param_key':[value] #params not to search
        #   }
        # check if necessary key pairs are passed
        
        for i in ['folder','save_model','save_prediction','save_metrics']:
            assert i in save_settings.keys()
            
        self.model_type=model_type
        self.model=None # tmp model for optuna / model load from file
        
        self.save_settings=save_settings
        self.save_path=os.path.join(save_settings['folder'],id)
        self.id=id
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        
        self.call_optuna=call_optuna
        if call_optuna==True:
            assert optuna_settings!=None
            assert optuna_params_dic!=None
        self.optuna_params_dic=optuna_params_dic
        self.optuna_settings=optuna_settings
        self.optuna_model_name=None
        
        self.call_one_trial=call_one_trial
        if call_one_trial==True:
            assert one_trial_settings!=None
            assert one_trial_params_dic!=None
        self.one_trial_params_dic=one_trial_params_dic
        self.one_trial_settings=one_trial_settings
        self.one_trial_model_name=None
        
        self.load_from_trained_model=load_from_trained_model
        if load_from_trained_model==True:
            assert load_settings!=None
            self.load_settings=load_settings
            self.load_from_model()
            
        self.load_from_checkpoint=load_from_checkpoint
        if load_from_checkpoint==True:
            assert load_checkpoint_settings!=None
            self.load_checkpoint_settings=load_checkpoint_settings
            self.load_from_checkpoints()
        
        
        self.predict_settings=predict_settings
        self.metric_settings=metric_settings
        

        
        self.optuna_study=None
        self.best_params=None
        self.best_model=None
        
        self.one_trial_model=None # model from run_one_trial
        
        ...
    def load_from_model(self):
        self.model=self.model_type.load(**self.load_settings)
        ...
        
    def load_from_checkpoint(self):
        self.model=self.model_type.load_from_checkpoint(**self.load_checkpoint_settings)
    # run one trial on designated params
    def run_one_trial(self):
        now=datetime.datetime.now()
        self.one_trial_model_name='-'.join([str(now.month),str(now.day),str(now.hour),str(now.minute),'one_trial'])
        
        self.one_trial_model=self.model_type(work_dir=self.save_path,
                                             model_name=self.one_trial_model_name, log_tensorboard=True,
                                             save_checkpoints=True,
                                             **self.one_trial_params_dic)
        #self.one_trial_model, optimizer = amp.initialize(self.one_trial_model, optimizer, opt_level='O1')

        self.one_trial_model.fit(**self.one_trial_settings)
        
    # run hyperparams searching
    def run_optuna(self):
        p=self.optuna_params_dic
        
        X_train=self.optuna_settings['X_train']
        y_train=self.optuna_settings['y_train']
        metric_cv=self.optuna_settings['metric_cv']
        n_trials=self.optuna_settings['n_trials']
        stop_threshold=self.optuna_settings['stop_threshold']
        direction=self.optuna_settings['direction']
        
        def objective(trial):
            params={}
            for i in p:
                if len(p[i])==1:
                    params.update({i:p[i][0]})
                elif len(p[i])==4:
                    #assert isinstance(p[i][1],p[i][0])
                    #assert isinstance(p[i][2],p[i][0])
                    if i =='lr':
                        params.update({'optimizer_kwargs':
                            {i:trial.suggest_float(i,p[i][1],p[i][2],log=p[i][3])}})
                    else:
                        if p[i][0]==int:
                            params.update({i:trial.suggest_int(i,p[i][1],p[i][2],log=p[i][3])})
                        elif p[i][0]==float:
                            params.update({i:trial.suggest_float(i,p[i][1],p[i][2],log=p[i][3])})
                        elif p[i][0]==bool:
                            params.update({i:trial.suggest_categorical(i,p[i][1],[p[i][2],p[i][3]])})
                        else:
                            Warning("Invalid param type!")
            model=RNNModel(**params)
            model.fit(**self.one_trial_settings)
            scores=cross_val_score(model, X_train, y_train,
                                   cv=KFold(n_splits=5,shuffle=True),
                                   scoring=metric_cv)
            report_cross_validation_scores(trial, scores)
            
            return scores.mean()
        
        def callback(study, trial):
            for ii, t in enumerate(study.trials):
                if t.value >= stop_threshold:
                    study.stop()
                    
        study = optuna.create_study(direction=direction)
        
        study.optimize(objective, n_trials=n_trials, gc_after_trial=True,
                   callbacks=[callback])
    
        print('Number of finished trials: {}'.format(len(study.trials)))
        print('Best trial:')
        trial = study.best_trial

        print('  Value: {}'.format(trial.value))
        print('  Params: ')

        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))
            
        self.optuna_study=study
        return study
    
    def refit_best_trial(self):
        assert self.optuna_study!=None
        best_trail=self.optuna_study.best_trial
        best_params=best_trail.params
        #best_params['tree_method']='gpu_hist'
        self.best_params=best_params
        model=self.model_type(**self.best_params)
        model.fit(self.optuna_setting['X_train'],self.optuna_setting['y_train'])
        self.best_model=model
        return model
    
    def save_one_trial_model(self):
        assert self.one_trial_model !=None
        model_path=os.path.join(self.save_path,self.one_trial_model_name,'trained_model')
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        model_name=self.one_trial_model_name
        model_name=os.path.join(model_path,model_name+'.pt')
        self.one_trial_model.save(model_name)
        
    def predict_on_one_trial_model(self):
        self.prediction=self.one_trial_model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,self.one_trial_model_name+'prediction.csv'))
        
    def predict_on_best_model(self):
        self.prediction=self.best_model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,self.optuna_model_name+'prediction.csv'))
        
    def predict_on_loaded_model(self):
        self.prediction=self.model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,'loaded_model_prediction.csv'))

    def cal_metrics(self):
        assert self.prediction!=None
        metrics_method_dic={
            'CV':metrics.coefficient_of_variation,
            'MAE':metrics.mae,
            'MAPE':metrics.mape,
            'OPE':metrics.ope,
            'RMSE':metrics.rmse,
            'MSE':metrics.mse,
            'MARRE':metrics.marre,
            'MASE':metrics.mase,
            'R2':metrics.r2_score,
            'SMAPE':metrics.smape,
        }
        metrics_dic={
            'start_time':self.prediction.time_index[0],
            'end_time':self.prediction.time_index[-1],
            'n':len(self.prediction.time_index),
        }
        
        for metric in metrics_method_dic.keys():
            try:
                if metric=='MASE':
                    value=metrics_method_dic[metric](self.metric_settings['series_pred_gt'],self.prediction,intersect=True,
                                                              insample=self.metric_settings['series_train'], m=96*7)
                    print({metric: value})
                    metrics_dic.update({metric: value})
                else:
                    value=metrics_method_dic[metric](self.metric_settings['series_pred_gt'],self.prediction,intersect=True)
                    print({metric: value})
                    metrics_dic.update({metric: value})
            except:
                print("Fail to calculate metric: {} of model {}".format(metric,self.id))
                
        metrics_df=pd.DataFrame([metrics_dic]).T
        metrics_df.to_csv(os.path.join(self.save_path,self.id+'_metrics.csv'))

    

# Probolistic RNN resembles DeepAR

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
import darts.utils.timeseries_generation as tg
from darts.datasets import AirPassengersDataset, EnergyDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood

In [12]:
# data preperation
bld_pd=pd.read_csv(r'/root/autodl-tmp/data/load_prediction_base/BLD_Sum.csv')
bld_pd.sort_values(by='DateTime')
bld_pd=bld_pd.drop(columns=['RealPower_before_scaling'])
bld=TimeSeries.from_dataframe(bld_pd,time_col="DateTime",freq="15min",fill_missing_dates=True)

bld=bld.drop_columns(['wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_main',
       'RealPower_-0d_0h'])

transformer = Scaler()

# data split
train_start=pd.Timestamp(2017,1,1,0,0)
train_end=pd.Timestamp(2018,12,31,23,45)

val_start=pd.Timestamp(2018,1,1,0,0)
val_end=pd.Timestamp(2018,12,31,23,45)

pred_start=pd.Timestamp(2019,1,1,0,0)
pred_end=pd.Timestamp(2019,12,31,23,45)

bld_train=bld[train_start:train_end]
bld_val=bld[val_start:val_end]
bld_pred=bld[pred_start:pred_end]

bld_trian=transformer.fit_transform(bld_train)
bld_val=transformer.fit_transform(bld_val)
#bld_trian=transformer.fit_transform(bld_train)

In [13]:
model_type=RNNModel
call_optuna=True
optuna_settings={
    'X_train':bld_train.drop_columns(['RealPower']),
    'y_train':bld_train['RealPower'],
    'metric_cv':'neg_mean_absolute_error',
    'n_trials':1,
    'stop_threshold':0.1,
    'direction':'maximize',
    
}
optuna_params_dic={
    'lr':[float, 1e-5, 1e-1, False],
    'model':['LSTM'],
    'hidden_dim':[int, 8, 256, False],
    'n_rnn_layers':[int,1,8,False],
    'batch_size':[int,8,96*2,False],
    'dropout':[float,0.01,0.5,False],
    'input_chunk_length':[96*7],
    'output_chunk_length':[96],
    'n_epochs':[int,1,2,False],
    'pl_trainer_kwargs':[{
      "accelerator": "gpu",
      "devices": [0]
    }],
    'random_state':[42],
}
call_one_trial=False
quantiles = [
    0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.4,0.5,
    0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99,
]

one_trial_settings={
    'series':bld_train['RealPower'],
    'future_covariates':bld_train.drop_columns(['RealPower']),
    #'val_series':bld_val['RealPower'],
    #'val_future_covariates':bld_val.drop_columns(['RealPower'])
}
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=1e-5,
    mode='min',
)

one_trial_params_dic={
    'optimizer_kwargs':{
        'lr':1e-4,
    },
    'pl_trainer_kwargs':{
        "callbacks": [my_stopper],
        "gradient_clip_val":0.1},
    'model':'LSTM',
    'hidden_dim':128,
    'n_rnn_layers':4,
    'batch_size':64,
    'dropout':0.2,
    'input_chunk_length':96*7,
    'output_chunk_length':96,
    #'loss_fn':torch.nn.GaussianNLLLoss(reduction='mean'),
    #'likelihood':GaussianLikelihood(),
    # QuantileRegression(
    #    quantiles=quantiles
    #),
    'n_epochs':10,
    'pl_trainer_kwargs':{
      "accelerator": "gpu",
      "devices": [0]
    },
    'random_state':42,
}
'''    
'likelihood':QuantileRegression(
        quantiles=quantiles
    ),'''
predict_settings={
    'n':96*365,
    'series':bld_train['RealPower'],
    'future_covariates':bld.drop_columns(['RealPower']),
    'n_jobs':-1,
    'num_samples':1
}
metric_settings={
    'series_pred_gt':bld_pred['RealPower'],
    'series_train':bld_train['RealPower'],
}
save_settings={
    'folder':r'/root/autodl-tmp/load_forecast/DeepAR(Prob_RNN)',
    'save_model':True,'save_prediction':True,'save_metrics':True
}
test=Model_base(
    id='try_optuna',
    model_type=RNNModel,
    call_optuna=True,
    optuna_settings=optuna_settings,
    optuna_params_dic=optuna_params_dic,
    call_one_trial=True,
    one_trial_settings=one_trial_settings,
    one_trial_params_dic=one_trial_params_dic,
    load_from_trained_model=False,
    load_settings=None,
    load_from_checkpoint=False,
    load_checkpoint_settings=None,
    predict_settings=predict_settings,
    metric_settings=metric_settings,
    save_settings=save_settings
)
test.run_optuna()
#test.save_one_trial_model()

[I 2023-10-11 02:34:43,859] A new study created in memory with name: no-name-7374ebe4-3101-42a6-84af-45a75117cc48
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 2.3 M 
4 | V             | Linear           | 229   
---------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.260     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
[W 2023-10-11 02:39:16,484] Trial 0 failed with parameters: {'lr': 0.05464417165024453, 'hidden_dim': 228, 'n_rnn_layers': 6, 'batch_size': 20, 'dropout': 0.413666970231875, 'n_epochs': 2} because of the following error: TypeError("Cannot clone object 'RNNModel(model=LSTM, hidden_dim=228, n_rnn_layers=6, dropout=0.413666970231875, training_length=24, optimizer_kwargs={'lr': 0.05464417165024453}, batch_size=20, input_chunk_length=672, output_chunk_length=96, n_epochs=2, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]}, random_state=42)' (type <class 'darts.models.forecasting.rnn_model.RNNModel'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.").
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1259/538616787.py", line 125, in objective
  

TypeError: Cannot clone object 'RNNModel(model=LSTM, hidden_dim=228, n_rnn_layers=6, dropout=0.413666970231875, training_length=24, optimizer_kwargs={'lr': 0.05464417165024453}, batch_size=20, input_chunk_length=672, output_chunk_length=96, n_epochs=2, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]}, random_state=42)' (type <class 'darts.models.forecasting.rnn_model.RNNModel'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
test.refit_best_trial()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
test.cal_metrics()

{'CV': 22.176691856419975}
{'MAE': 13.980581074241446}
{'MAPE': 20.528752670935834}
{'OPE': 2.595183476391746}
{'RMSE': 16.127937103539352}
{'MSE': 260.11035521572137}
{'MARRE': 19.473432787052577}
{'MASE': 3.002026990660021}
{'R2': -0.013884527538935876}
{'SMAPE': 19.249988436986946}
